# Top 5 Quotes dummy model

In [1]:
import pandas as pd
import itertools
import re
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import string
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [2]:
file = "../raw_data/quotes.csv"
quotes = pd.read_csv(file, decimal=",")
quotes.head(3)

,quote,author,category
0,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,"attributed-no-source, best, life, love, mistak..."
1,You've gotta dance like there's nobody watchin...,William W. Purkey,"dance, heaven, hurt, inspirational, life, love..."
2,You know you're in love when you can't fall as...,Dr. Seuss,"attributed-no-source, dreams, love, reality, s..."


In [3]:
tags_list = quotes.category.to_list()
number_of_tags = []
for i in tags_list:
    number_of_tags.append(len(str(i).split(',')))

In [4]:
quotes['quotes_length'] = quotes['quote'].copy()
for index, row in quotes.iterrows():
    quotes.loc[index, "quotes_length"] = len(str(row['quotes_length']).split(' '))

## Cleaning data: tags are all lowercase, '-' were replaced with ',', duplicates were removed

In [5]:
for index, row in quotes.iterrows():
    quotes.loc[index, "category"] = str(row['category']).lower().replace('-',', ')

In [6]:
quotes['list_tags'] = quotes['category'].copy()
for index,row in quotes.iterrows():
    quotes.loc[index, 'count_tags'] = len(str(row['list_tags']).split(','))
quotes.count_tags.describe()

count    499709.000000
mean          6.684064
std          10.532606
min           1.000000
25%           2.000000
50%           3.000000
75%           7.000000
max         568.000000
Name: count_tags, dtype: float64

In [7]:
for index,row in quotes.iterrows():
    quotes.at[index, 'list_tags'] = str(row['list_tags']).split(',')

In [8]:
for index,row in quotes.iterrows():
    quotes.at[index, 'list_tags'] = str(set(row['list_tags']))

In [9]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
quotes['list_tags'] = quotes['list_tags'].apply(remove_punctuations)

In [10]:
vectorizer = TfidfVectorizer()
tags_vectorized = vectorizer.fit_transform(quotes['list_tags'])
dummy_tags = LatentDirichletAllocation(n_components=5)
lda_vectors = dummy_tags.fit_transform(tags_vectorized)

In [11]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])
        

print_topics(dummy_tags, vectorizer.fit(quotes['list_tags']))

Topic 0:
[('women', 2003.2581827849167), ('religion', 1908.0676301053654), ('nature', 1890.6100338399694), ('science', 1772.937646835534), ('love', 1766.3299356477967), ('human', 1652.4255568335732), ('men', 1581.2149618745252), ('money', 1568.487889399725), ('god', 1542.7426505656852), ('life', 1542.3320486167172)]
Topic 1:
[('life', 7154.573897807301), ('self', 4631.441743724187), ('inspirational', 4120.105029027829), ('success', 3415.9149167223995), ('quotes', 3398.7426084176686), ('philosophy', 2875.738313595533), ('love', 2589.2768360706864), ('you', 2444.3078771890036), ('living', 2307.287000244068), ('people', 2117.3967304893235)]
Topic 2:
[('inspirational', 5670.415183345447), ('love', 4124.27367091982), ('quotes', 3928.36471879855), ('writing', 3350.0344684543275), ('life', 2982.9977185728844), ('god', 2662.9377959705625), ('books', 2452.925589008916), ('wisdom', 2282.7205099032053), ('reading', 2108.018266105808), ('faith', 2094.1382041387405)]
Topic 3:
[('happiness', 2628.03

In [15]:
lda_vectors[0]

array([0.04701307, 0.63154942, 0.22661882, 0.04728886, 0.04752983])

In [13]:
quotes.iloc[-1] = ['me, Paris, selfie, singing, book','image','image','5','image','1']

In [ ]:
picture_vectorized = vectorizer.transform(quotes.iloc[-1])
lda_vectors_self = dummy_tags.transform(picture_vectorized)
for index,row in quotes.iterrows():
    quotes['topic'] = quotes.quote.copy()
    quotes.at[index, 'topic'] = np.where(lda_vectors[0] == max(lda_vectors[0]))[0]

In [ ]:
quotes.head()